# Asset Management
## Introduction
In this module we will consider how to construct an optimal portfolio. This means that we will identify the portfolio of assets that makes an optimal tradeoff between risk and expected return. In previous modules, we have discussed the notion of constructing portfolios via two decisions, these being the asset allocation and security selection decision. Mathematically speaking, we will see that these decisions are identical. A such, we investigate the problem of constructing an optimal portfolio from a set of risky securities. The key idea is to understand how diversification affects the risk of the portfolio and hence how one can optimally diversify by carefully selecting the portfolio weights. Next, we return to the factor structure we examined in module 8 and consider how we address the same problem, how to select the optimal portfolio, when returns are assumed to have a factor structure.

## Diversification and risk
The risk and diversification are intimately linked. To provide intuition, you can think of where the risk of a single asset comes from. Some of that risk come from the conditions of the general economy while the rest may come from the specific industry that firm operates in, or even from the specific choices management make. By investing in many assets, we are unlikely to be able to eliminate fluctuations due to the risk in the broader economy because all assets will be similarly affected. However, by investing in many assets, we find that their fluctuations in value due to firm and industry specific sources of risk do not all move in the same direction. Hence, when considering the portfolio of these assets, the fluctuations due to firm/industry sources tend to cancel each other out (there are ups and downs that average out to smaller fluctuations than individual assets have). This is the principle of diversification. The mathematical reason for this is that the fluctuations due to firm/industry sources are relatively uncorrelated. They tend not to move together. In fact, we will use the notion of correlation to develop a measure of risk that can be used to build optimal portfolios.

## Two risky assets
To understand how the principle of diversification works, we will consider a simple example consisting of two risk assets only. We will call these assets $A$ and $B$ and examine how portfolio risk changes as we alter portfolio weights. To this end, we define risk as the standard deviation of portfolio returns. This quantity can be computed from the individual assets returns standard deviations as well as the covariance between the assets returns. Recall that covariance and correlation ($\rho$) are related by the formula
\begin{align}
    \rho_{A,B} = \frac{Cov(r_{A},r_{B})}{\sigma_{A}\sigma_{B}}
\end{align}
Hence correlation is simply a rescaled version of covariance. This makes correlation easier to interpret but from a mathematical perspective, it will often be easier to use covariance. The key point is that they are the same thing, just expressed on different scales. The formula for the variance (standard deviation squared) of a portfolio of assets $A$ and $B$ with respective weights $w_{A}$ and $w_{B}$ is
\begin{align}
    \sigma_{P}^{2} = w_{A}\sigma_{A}^{2} + w_{B}\sigma_{B}^{2} + 2w_{A}w_{B}Cov(r_{A}, r_{B})
\end{align}
In order for this to represent a portfolio with full investment of wealth, we require the weights sum to 1
\begin{align}
    w_{A} + w_{B} = 1 \quad \Rightarrow \quad w_{A} = 1-w_{B}
\end{align}
This means we can write the variance of a portfolio of two assets in terms of a single variable, $w_{A} = w$.
\begin{align}
    \sigma_{P}^{2} = w^{2}\sigma_{A}^{2} + (1-w)^{2}\sigma_{B}^{2} + 2w(1-w)Cov(r_{A}, r_{B})
    \label{eq:two asset portfolio var} \tag{1}
\end{align}


### Python example
The code below implements the equation for portfolio risk as a function of the weight $w$. You can adjust the level of correlation by moving the slider to see how portfolio risk changes at different levels of asset correlation. Notice that if $|\rho|<1$ then we can always produce a portfolio that has less risk than either of the constituent assets. What do you notice when $\rho = 1$? What about when $\rho = -1$?

In [7]:
import numpy as np
from bokeh.layouts import column, row
from bokeh.models import CustomJS, Slider, HoverTool
from bokeh.plotting import ColumnDataSource, figure, show#, output_file
# Make Bokeh Push push output to Jupyter Notebook.
from bokeh.io import push_notebook, show, output_notebook, reset_output
reset_output()
output_notebook()

sigA = 0.2
sigB = 0.35
corrStart = 0.4
w = np.linspace(-1, 2, 500)
sigP = np.sqrt(w**2*sigA**2 + (1-w)**2*sigB**2 + 2*w*(1-w)*corrStart*sigA*sigB)

source = ColumnDataSource(data=dict(x=w, y=sigP))

plot = figure(x_range=(-1, 2), y_range=(0, 0.7), plot_width=400, plot_height=400, title="Correlation and Diversification", x_axis_label="w", y_axis_label="Portfolio Risk")

plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6, legend_label="Portfolio")
plot.circle(x=1, y=sigA, size=8, fill_color="blue", legend_label="Asset A")
plot.circle(x=0, y=sigB, size=8, fill_color="green", legend_label="Asset B")
plot.legend.location = "bottom_left"
plot.legend.background_fill_alpha = 0
plot.legend.border_line_alpha  = 0
corr_slider = Slider(start=-1, end=1, value=corrStart, step=.1, title="Correlation")

callback = CustomJS(args=dict(source=source, corr=corr_slider, sA = sigA, sB = sigB),
                    code="""
    const data = source.data;
    const C = corr.value;
    const x = data['x']
    const y = data['y']
    for (var i = 0; i < x.length; i++) {
        y[i] = Math.sqrt(x[i]**2*sA**2 + (1-x[i])**2*sB**2 + 2*x[i]*(1-x[i])*C*sA*sB);
    }
    source.change.emit();
""")

corr_slider.js_on_change('value', callback)

layout = column(
    column(corr_slider),
    plot,
)

show(layout)

Loading BokehJS ...

If $\rho = 1$, then the equation for portfolio risk reduces to
\begin{align}
    \sigma_{P}^{2} &= w_{A}^{2}\sigma_{A}^{2} + w_{B}^{2}\sigma_{B}^{2} + 2w_{A}w_{B}\sigma_{A}\sigma_{B}\\
    &= (w_{A}\sigma_{A} + w_{B}\sigma_{B})^{2}\\
    \sigma_{P} &= w_{A}\sigma_{A} + w_{B}\sigma_{B}
\end{align}
and hence portfolio risk is simply a weighted average of the two constituents’ assets risks. Similarly, if $\rho = -1$, we can show that
\begin{align}
    \sigma_{P} &= w_{A}\sigma_{A} - w_{B}\sigma_{B}
\end{align}

## The opportunity set
At this stage, we have limited our analysis to portfolio risk and have seen that, by combining assets with imperfect correlations, we can produce a portfolio that has lower risk than the constituent assets could provide on their own. But when selecting portfolios, we are also concerned with the rewards we will get. Hence, we now include the expected return in our analysis. This will produce a subset of the risk-reward space which shows what kinds of investment opportunities are available to investors. We call this collection of portfolios the opportunity set. We begin with an analysis of two-asset portfolios.

Recall that the expected return of a two-asset portfolio is given by
\begin{align}
    E(r_{P}) &= w_{A}E(r_{A}) + w_{B}E(r_{B})\\
    &= wE(r_{A}) + (1-w)E(r_{B})
\end{align}
Taking this expression together with equation \eqref{eq:two asset portfolio var} produces a curve (hyperboala) in the risk-reward space.

### Python example
The code below produces a plot of the two-asset opportunity set (just a line in this case). All possible portfolios lie on the lie, the shape of which is determined by the correlation between the assets.

In [8]:
reset_output()
output_notebook()

ErA = 0.1
ErB = 0.13
sigA = 0.2
sigB = 0.35
corrStart = 0.4
w = np.linspace(-1, 2, 500)
ErP = w*ErA + (1-w)*ErB
sigP = np.sqrt(w**2*sigA**2 + (1-w)**2*sigB**2 + 2*w*(1-w)*corrStart*sigA*sigB)

source = ColumnDataSource(data=dict(x=sigP, y=ErP))

plot = figure(x_range=(0, 0.7), y_range=(0.05, 0.2), plot_width=400, plot_height=400, title="The Opportunity Set", x_axis_label="Risk", y_axis_label="Expected Return")

plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6, legend_label="Opportunity set")
plot.circle(x=sigA, y=ErA, size=8, fill_color="blue", legend_label="Asset A")
plot.circle(x=sigB, y=ErB, size=8, fill_color="green", legend_label="Asset B")
plot.legend.location = "top_left"
plot.legend.background_fill_alpha = 0
plot.legend.border_line_alpha  = 0

corr_slider = Slider(start=-1, end=1, value=corrStart, step=.1, title="Correlation")

callback = CustomJS(args=dict(source=source, corr=corr_slider, w = w, rA = ErA, rB = ErB, sA = sigA, sB = sigB),
                    code="""
    const data = source.data;
    const C = corr.value;
    const x = data['x']
    const y = data['y']
    for (var i = 0; i < x.length; i++) {
        x[i] = Math.sqrt(w[i]**2*sA**2 + (1-w[i])**2*sB**2 + 2*w[i]*(1-w[i])*C*sA*sB);
        y[i] = w[i]*rA + (1-w[i])*rB
    }
    source.change.emit();
""")

corr_slider.js_on_change('value', callback)

layout = column(
    column(corr_slider),
    plot,
)

show(layout)

Loading BokehJS ...

## Adding in a risk-free asset
We now consider how the scenario changes when a risk-free security (such as a T-Bill) is included in the analysis with the two risky assets. The key to understanding this problem is that the risk-free asset has no variance and no covariance with the risky assets returns. This means that if we combine the risk-free asset with a portfolio $P$ (which at present is made from two risky assets only) then the expected return and variance of this portfolio, $Q$, is given by
\begin{align}
    E(r_{Q}) &= w_{r_{f}}r_{f} + (1-w_{r_{f}})E(r_{P})\\
    \sigma_{Q}^{2} &= w_{r_{f}}^{2}\underbrace{\sigma_{r_{f}}^{2}}_{=0} + (1-w_{r_{f}})^{2}\sigma_{P}^{2}\\
    \sigma_{Q} &= (1-w_{r_{f}})\sigma_{P}
\end{align}
Hence, the risk-return combinations are straight lines in the risk-reward space (not hyperbolas as with two risky assets). The straight line will extend from the risk-rate to the selected portfolio $P$. This result implies that when a risk-free asset is available, there is one single risky portfolio that is preferred above all others, regardless of any investor preferences regarding risk and return. This is because any rational investor will prefer greater levels of return and lower levels of risk. In risk-reward space, this is the portfolio with the highest Sharpe ratio (greatest reward per unit of risk). Hence, we can identify the optimal risky portfolio (in a two-asset scenario) by solving the problem
\begin{align}
    \max_{w}\frac{E(r_{P}) - r_{f}}{\sigma_{P}} = \frac{wE(r_{A}) + (1-w)E(r_{B}) - r_{f}}{\sqrt{w^{2}\sigma_{A}^{2} + (1-w)^{2}\sigma_{B}^{2} + 2w(1-w)Cov(r_{A}, r_{B})}}
\end{align}
A direct application of calculus methods can solve this problem, though we leave how to do this to other, more advanced, courses.

In [9]:
reset_output()
output_notebook()

rf = 0.05
ErA = 0.1
ErB = 0.13
sigA = 0.2
sigB = 0.35
corrStart = 0.4
wp = np.linspace(-1, 2, 500)
ErP = wp*ErA + (1-wp)*ErB
sigP = np.sqrt(wp**2*sigA**2 + (1-wp)**2*sigB**2 + 2*wp*(1-wp)*corrStart*sigA*sigB)


# Compute the optimal portfolio
wA = (ErA*sigB**2 - ErB*corrStart*sigA*sigB)/(ErA*sigB**2 + ErB*sigA**2 - (ErA + ErB)*corrStart*sigA*sigB)
wB = 1-wA
ErOpt = wA*ErA + wB*ErB
sigOpt = np.sqrt(wA**2*sigA**2 + wB**2*sigB**2 + 2*wA*wB*corrStart*sigA*sigB)

# Risk free + optimal portfolio
wq = np.linspace(-1, 1, 500)
ErQ = wq*rf + (1-wq)*ErOpt
sigQ = (1-wq)*sigOpt

sourceP = ColumnDataSource(data=dict(x=sigP, y=ErP))
sourceQ = ColumnDataSource(data=dict(x=sigQ, y=ErQ))

plot = figure(x_range=(0, 0.7), y_range=(0.05, 0.2), plot_width=400, plot_height=400, title="The Optimal Portfolio", x_axis_label="Risk", y_axis_label="Expected Return")

plot.line('x', 'y', source=sourceP, line_width=3, line_alpha=0.6, legend_label="Op. Set")
plot.line('x', 'y', source=sourceQ, line_width=3, line_alpha=0.6, line_color="red", legend_label="CAL", line_dash="dashed")
plot.circle(x=sigA, y=ErA, size=8, fill_color="blue", legend_label="Asset A")
plot.circle(x=sigB, y=ErB, size=8, fill_color="green", legend_label="Asset B")
plot.circle(x=sigOpt, y=ErOpt, size=8, fill_color="red", legend_label="Opt Pflio")
plot.legend.location = "top_left"
plot.legend.background_fill_alpha = 0
plot.legend.border_line_alpha  = 0
layout = column(plot)
show(layout)

Loading BokehJS ...

## Markowitz Portfolio Optimization Model
What we have studied thus far has been restricted to two risky assets only. This was done for mathematical simplicity. In particular, the expression for the variance of a portfolio is straightforward (see equation $\eqref{eq:two asset portfolio var}$). But what happens when we have more than two risky assets? One of the simplifications of dealing with two risky assets only is that the opportunity set is also the set of optimal portfolios. This is because we cannot create a portfolio from the two assets that has a different risk-return combination. However, when we move to three or more risky assets, this is no longer the case. The opportunity set and the set of minimum variance (risk) portfolios are not the same. Hence, we must mathematically determine the set of portfolios that have the least risk for a given return. This is the problem that Harry Markowitz solved in 1952 and for which he would later win the Nobel in economics.

We will not go into the mathematical details of this procedure, but instead outline the results that were found. First, the opportunity set is the interior of a hyperbola in standard deviation-expected return (risk-reward) space. The edge of this set represents the minimum variance portfolios. As their name suggests, these portfolios have the lowest risk for a given level of return. However, only part of this minimum variance set is desirable. This is because for a given level of risk (standard deviation) there are two possible expected returns that can be earned. Any rational investor will select the portfolio with the higher level of return for a given level of risk. This leads to the set of portfolios that are deemed optimal among all risky portfolios and we call this set the *efficient frontier*.

Watch the video to see an in depth explanation of these concepts.

## The separation property
In the two risky asset scenario, we found that there is a single risky portfolio that all investors will agree upon if a risk-free asset is available. This result was driven by the fact that the risk-free asset has no variance or covariance with any risk assets or portfolios made from them. This result is unchanged if we add more risky assets to the problem. Hence, we can separate the investment problem again in this more complicated scenario. First, we identify the optimal risky portfolio by finding the portfolio of risky assets only that has the highest Sharpe ratio. Next, we determine how much wealth to split between the optimal risky portfolio and the risk-free asset. Investors with higher levels of risk aversion (prefer lower levels of risk) will put more of their wealth in the risk-free asset to minimize the amount of risk they are exposed to. Investors with lower levels of risk aversion will place more of their wealth in the optimal risky portfolio to earn higher returns. Some investors with very low risk aversion may even borrow money at the risk-free rate and investor more than 100\% of their wealth in the optimal risky portfolio in an effort to earn larger returns. This of course comes with significant risk. We call the separate tasks of finding the optimal risky portfolio, then combining that portfolio with the risk-free asset the separation property. Namely, we ca treat these two tasks as separate operations and still get the optimal portfolio.

Watch the video for a thorough explanation


## Diversification
We are now able to understand where the power of diversification comes from. Recall that the variance of a portfolio can be written as
\begin{align}
    \sigma_{P}^{2} = \sum_{i=1}^{n}\sum_{j=1}^{n}w_{i}w_{j}Cov(r_{i}, r_{j})
\end{align}
where $w_{i}$ is the weight in asset $i$ and $i=1, 2, \ldots, n$. Note that if $i=j$, we have asset $i$s variance. Splitting these terms out gives
\begin{align}
    \sigma_{P}^{2} = \underbrace{\sum_{i=1}^{n}w_{i}^{2}\sigma_{i}^{2}}_{n \textrm{ terms}} + \underbrace{\sum_{i=1}^{n}\sum_{j=1, j\neq i}^{n}w_{i}w_{j}Cov(r_{i}, r_{j})}_{n(n-1) \textrm{ terms}}
\end{align}
To simplify, let’s assume an equally weighted portfolio and hence $w_{i} = \tfrac{1}{n}$. Define the average asset variance as
\begin{align}
    \bar{\sigma}^{2} = \frac{1}{n}\sum_{i=1}^{n}\sigma_{i}^{2}
\end{align}
and the average asset covariance as
\begin{align}
    \overline{Cov} = \frac{1}{n(n-1)}\sum_{i=1}^{n}\sum_{j=1, j\neq i}^{n}w_{i}w_{j}Cov(r_{i}, r_{j})
\end{align}
Then, in this simplified case, portfolio variance can be written as
\begin{align}
    \sigma_{P}^{2} = \frac{1}{n}\bar{\sigma}^{2} + \frac{n-1}{n}\overline{Cov}
\end{align}
Imagine for the moment that all assets are independent. Then $\overline{Cov}=0$ and we can see that by increasing the number of assets in our portfolio ($n$), the risk will decrease. However, note that the coefficient of $\overline{Cov}$ doesn't vanish as we increase $n$. This shows that the covariance between assets (equivalently correlation) places a limit to how much we can diversify.


## Asset allocation and security selection
We have spoken in earlier modules about the asset allocation and security selection decisions. It appears that these are now redundant as we can use Markowitz optimal portfolio theory to just identify the best portfolio. While theoretically true, this is not how fund managers operate in practice. Typically, an optimization is performed over the broad asset classes. Once these allocations have been established, another Markowitz style optimization is performed across the individual securities within each asset class to produce the optimal portfolios for each asset class. The main reason for this approach is organizational efficiency. There is a different set of skills required to be an expert in fixed income vs equities and as such experts within each asset class construct their ideal portfolios. Once this is established it is then a much easier task to determine how capital should be allocated across broad asset classes rather than trying to combine all the myriad of assets together in a single optimization.

## The single index model
When performing Markowitz optimization, many inputs are required. Specifically, we need
* $n$ inputs for $E(r_{i})$
* $n$ inputs for $\sigma_{i}^{2}$
* $\tfrac{1}{2}n(n-1)$ inputs for $Cov(r_{i},r_{j})$
Additionally, the covariances must be chosen in a consistent manner, otherwise this can create logical inconsistencies and the entire procedure will fail. Though producing all the required estimates is possible, the single index model (SIM) provides a convenient way to mitigate some of these issues. Recall that the SIM assumes that returns are generated via the expression
\begin{align}
    r_{i} = E(r_{i}) + \beta_{i} r_{M} + e_{i}
\end{align}
where $r_{M}$ is the return on a market portfolio. It was also assumed that the market portfolios return was independent of all other sources of risk (the $e_{i}$) and that all idiosyncratic sources of risk were also independent from one another. Under these assumptions, the variance of asset $i$s return is given by
\begin{align}
    \sigma_{i}^{2} = \beta_{i}^{2}\sigma_{M}^{2} + \sigma_{e_{i}}^{2}
\end{align}
The risk of a security is composed of a systematic component ($\beta_{i}^{2}\sigma_{M}^{2}$) and an idiosyncratic component ($\sigma_{e_{i}}^{2}$). Looking at covariance, we have that
\begin{align}
    Cov(r_{i},r_{j}) &= Cov(\beta_{i} r_{M} + e_{i},\beta_{j} r_{M} + e_{j})\\
    &= Cov(\beta_{i} r_{M}, \beta_{j} r_{M}) + \underbrace{Cov(e_{i}, \beta_{j} r_{M})}_{=0} + \underbrace{Cov(\beta_{i} r_{M}, e_{j})}_{=0} + \underbrace{Cov(e_{i},e_{j})}_{=0}\\
    &= \beta_{i}\beta_{j}\sigma_{M}^{2}
\end{align}

Recall from prior modules that when estimating an index model, we must perform a regression which is typically written as
\begin{align}
    R_{i} = \alpha_{i} + \beta_{i} R_{M} + \epsilon_{i}
\end{align}
and hence if we know all the statistical characteristics of the quantities in the above regression, we can perform our portfolio optimization. In this case we would require
* $n$ estimates of $\alpha_{i}$
* $n$ estimates of $\beta_{i}$
* $n$ estimates of $\sigma_{\epsilon_{i}}^{2}$
* 1 estimate of $E(r_{M})$
* 1 estimates of $\sigma_{M}^{2}$
This gives a total of $3n + 2$ estimates which grows linearly with $n$ (rather than quadratically for the full model) and as such is much less computationally challenging to implement. Additionally, using the SIM ensures that all covariances will be computed consistently with respect to each other.


## The Treynor-Black model
The Treynor-Black model is a special case of the Markowitz model with a risk-free asset and as such is designed to identify the unique optimal risky portfolio. However, it makes two assumptions which allows the analysis. First, it assumes the SIM specification for returns. Second, it assumes that the market portfolio can be bought and sold like a regular asset. This assumption is not far from reality as there are numerous funds that provide access to an exchange traded fund (ETF) that tracks a market index. Via these ETFs, investors can essentially purchase the market portfolio in a single, straightforward transaction.

The reason the inclusion of a market tracking portfolio changes things is because we are now interested in whether we can beat the market by taking larger positions in assets with positive alphas, short positions in assets with negative alphas, and maintaining a position in the market portfolio to keep risk down through diversification. Treynor and Black solved this problem and outlined the economic intuition behind each of the steps. It was the first formal model of active portfolio management as it advocates for deviating from market capitalization weights (a passive approach) if the alphas of individual assets can be leveraged to produce an outperforming portfolio. We now outline the steps required to build the optimal portfolio.

First, let the market portfolio be the $n+1$ asset. The other assets (individual stocks) are listed as asset $i=1, 2, \ldots, n$. Note that $\alpha_{n+1} = \alpha_{M}=0$, $\beta_{n+1}=\beta_{M}=1$ and $\sigma_{e_{n+1}}=\sigma_{e_{M}}=0$ by definition. Then, the aim of the portfolio optimization is to identify the weights $w_{i}$, $i=1, 2, \ldots, n+1$ where we wish to maximize
\begin{align}
    \max_{w_{i}}\frac{E(R_{p})}{\sigma_{p}}
\end{align}
where
\begin{align}
    E(R_{p}) &= \alpha_{p} + E(R_{M})\beta_{p} = \sum_{i=1}^{n+1}w_{i}\alpha_{i} + E(R_{M})\sum_{i=1}^{n+1}w_{i}\beta{i}\\
    \sigma_{p} &= \beta_{p}^{2}\sigma_{M}^{2} + \sigma_{e_{p}}^{2} = \sqrt{\sigma_{M}^{2}\left(\sum_{i=1}^{n+1}w_{i}\beta_{i}\right)^{2} + \sum_{i=1}^{n+1}w_{i}^{2}\sigma_{e_{i}}^{2}}
\end{align}

If we were interested only in diversification, we would just hold the market portfolio, but since we wish to produce the best expected return per unit of risk, we will take larger positions in assets with high alphas and vice versa. It turns out that to build the optimal portfolio, we construct an active portfolio, $A$, and combine this with a passive position (just the market portfolio). We first identify the allocation to the active and passive portfolios, then we will compute the individual active weights. Lets say that the beta of our active portfolio is 1. Then the active weight would be given by
\begin{align}
    w_{A}^{0} = \frac{\frac{\alpha_{A}}{\sigma_{e_{A}}^{2}}}{\frac{E(R_{M})}{\sigma_{M^{2}}}}
\end{align}
This fraction represents the contribution the active portfolio makes to expected return per unit of risk and we moderate this with the equivalent ratio for the market portfolio. If the beta of the active position is not 1, then we need to adjust this ratio. Say the $\beta_{A} > 1$. Then this would mean that the active portfolio and passive portfolio are relatively highly correlated and hence that we get less of a diversification benefit from the active portfolio. We would hence reduce our position in the active portfolio. The way we scale the previous weight is
\begin{align}
    w_{A}^{*} = \frac{w_{A}^{0}}{1 + (1-\beta_{A})w_{A}^{0}}
\end{align}
Note that if $\beta_{A} = 1$ then $w_{A}^{*} = w_{A}^{0}$. Of course the allocation to the market portfolio is simply $w_{M}^{*} = 1 - w_{A}^{*}$ as portfolio weights must sum to 1.

In order to compute the above equations, we need to known the individual weights in the active portfolio. These are computed as the normalized information ratio (using variance instead of standard deviation though). Namely, we compute
\begin{align}
    w_{i}^{*} = w_{A}^{*}\frac{\frac{\alpha_{i}}{\sigma_{e_{i}}^{2}}}{\sum_{i=1}^{n}\frac{\alpha_{i}}{\sigma_{e_{i}}^{2}}}
\end{align}
Note that the above expression will sum to $w_{A}^{*}$ by construction. Once we have these weights, we can compute the alpha and idiosyncratic variance of the active portfolio and use this to determine how to allocate our wealth across the active and passive portfolios.


In [10]:
import pandas as pd 
import statsmodels.api as sm

# Load the .csv file which has been saved locally.
# Column separators are denoted with "," and the headers for each column are on line 0.
Data = pd.read_csv(r"E:\Dropbox\Teaching\FINC6001\Module 11 - Portfolio Theory\Data\Module_11_industry_data.csv", sep = ",", header = 0)
npData = Data.to_numpy()

# Store the market reutn as variable X
X = npData[:,11]
# Compute the mean and variance of X
ERM = np.mean(X)
VarRM = np.var(X)
# Add the regression constant in
X = sm.add_constant(X)
# Store the risk free rate
RF = npData[:,12]
# Preallocate arrays for alpha, beta and idiosyncratic variance
alpha = np.zeros(10)
beta = np.zeros(10)
Var_e = np.zeros(10)
# Loop over each industry and perform the SIM regression, storing results in the arrays we just made
for i in range(1,11):
    Y = npData[:,i] - RF
    RegModel = sm.OLS(Y, X)
    RegResults = RegModel.fit()
    alpha[i-1] = RegResults.params[0]
    beta[i-1] = RegResults.params[1]
    Var_e[i-1] = RegResults.mse_resid
    
# Compute the individual active weights
wAi = (alpha/Var_e)/np.sum(alpha/Var_e)
# Compute the active portfolio alpha
alphaA = np.sum(wAi*alpha)
Var_eA = np.sum(wAi**2*Var_e)
# Compute all other weights
wA0 = (alphaA/Var_eA)/(ERM/VarRM)
betaA = np.sum(wAi*beta)
wAopt = wA0/(1 + (1-betaA)*wA0)
wMopt = 1-wAopt

# STore the industry labels in a new variable
Industries = Data.columns[1:11]
# Create dataframe to display the individual industry weights (wAi)
ActiveWeights = pd.DataFrame(wAi,Industries)
# Print the results
print('The allocations are ' + str(round(wAopt,3)) + ' to the active portfolio and ' + str(round(wMopt,3)) + ' to the passive portfolio.')
print('The active weights in each index are:')
ActiveWeights

The allocations are -1.115 to the active portfolio and 2.115 to the passive portfolio.
The active weights in each index are:


,0
NoDur,0.784121
Durbl,-0.018353
Manuf,0.551941
Enrgy,0.529391
HiTec,-1.334986
Telcm,0.416347
Shops,-0.354013
Hlth,-0.031826
Utils,-0.056346
Other,0.513724
